# "Training spaCy 3 on IDT"
> "Not going well so far"

- toc: false
- branch: master
- comments: true
- categories: [spacy, idt]


In [1]:
%%capture
!pip install -U pip setuptools wheel
!pip install thinc --pre
!pip install -U spacy spacy-lookups-data

In [24]:
!python -m spacy project clone pipelines/tagger_parser_ud

✔ Cloned 'pipelines/tagger_parser_ud' from explosion/projects
/content/tagger_parser_ud
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /content/tagger_parser_ud


In [25]:
%%writefile tagger_parser_ud/project.yml
title: "Part-of-speech Tagging & Dependency Parsing (Universal Dependencies)"
description: "This project template lets you train a part-of-speech tagger, morphologizer and dependency parser from a [Universal Dependencies](https://universaldependencies.org/) corpus. It takes care of downloading the treebank, converting it to spaCy's format and training and evaluating the model. The template uses the [`UD_English-EWT`](https://github.com/UniversalDependencies/UD_English-EWT) treebank by default, but you can swap it out for any other available treebank. Just make sure to adjust the `lang` and treebank settings in the variables below. Use `xx` for multi-language if no language-specific tokenizer is available in spaCy. Note that multi-word tokens will be merged together when the corpus is converted since spaCy does not support multi-word token expansion."

# Variables can be referenced across the project.yml using ${vars.var_name}
vars:
  config: "default"
  lang: "ga"
  treebank: "UD_Irish-IDT"
  train_name: "ga_idt-ud-train"
  dev_name: "ga_idt-ud-dev"
  test_name: "ga_idt-ud-test"
  package_name: "ud_ga_idt"
  package_version: "0.0.0"
  gpu: -1

# These are the directories that the project needs. The project CLI will make
# sure that they always exist.
directories: ["assets", "corpus", "training", "metrics", "configs", "packages"]

assets:
  - url: "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ga.300.vec.gz"
    dest: "assets/vectors.vec.gz"
    description: "FastText vectors"
  - dest: "assets/${vars.treebank}"
    git:
      repo: "https://github.com/UniversalDependencies/${vars.treebank}"
      branch: "master"
      path: ""

workflows:
  all:
    - preprocess
    - vectors
    - train
    - evaluate
    - package

commands:
  - name: preprocess
    help: "Convert the data to spaCy's format"
    script:
      - "mkdir -p corpus/${vars.treebank}"
      - "python -m spacy convert assets/${vars.treebank}/${vars.train_name}.conllu corpus/${vars.treebank}/ --converter conllu --n-sents 10 --merge-subtokens --morphology"
      - "python -m spacy convert assets/${vars.treebank}/${vars.dev_name}.conllu corpus/${vars.treebank}/ --converter conllu --n-sents 10 --merge-subtokens --morphology"
      - "python -m spacy convert assets/${vars.treebank}/${vars.test_name}.conllu corpus/${vars.treebank}/ --converter conllu --n-sents 10 --merge-subtokens --morphology"
      - "mv corpus/${vars.treebank}/${vars.train_name}.spacy corpus/${vars.treebank}/train.spacy"
      - "mv corpus/${vars.treebank}/${vars.dev_name}.spacy corpus/${vars.treebank}/dev.spacy"
      - "mv corpus/${vars.treebank}/${vars.test_name}.spacy corpus/${vars.treebank}/test.spacy"
    deps:
      - "assets/${vars.treebank}/${vars.train_name}.conllu"
      - "assets/${vars.treebank}/${vars.dev_name}.conllu"
      - "assets/${vars.treebank}/${vars.test_name}.conllu"
    outputs:
      - "corpus/${vars.treebank}/train.spacy"
      - "corpus/${vars.treebank}/dev.spacy"
      - "corpus/${vars.treebank}/test.spacy"

  - name: vectors
    help: "Convert, truncate and prune the vectors."
    script:
      - "python -m spacy init vectors ga assets/vectors.vec.gz corpus/ga_vectors -n ga_fasttext_cc_vectors_md"
    deps:
      - "assets/vectors.vec.gz"
    outputs:
      - "corpus/ga_vectors"

  - name: train
    help: "Train ${vars.treebank}"
    script:
      - "python -m spacy train configs/${vars.config}.cfg --output training/${vars.treebank} --gpu-id ${vars.gpu} --paths.train corpus/${vars.treebank}/train.spacy --paths.dev corpus/${vars.treebank}/dev.spacy --paths.vectors corpus/ga_vectors --nlp.lang=${vars.lang}"
    deps:
      - "corpus/${vars.treebank}/train.spacy"
      - "corpus/${vars.treebank}/dev.spacy"
      - "configs/${vars.config}.cfg"
      - "corpus/ga_vectors"
    outputs:
      - "training/${vars.treebank}/model-best"

  - name: evaluate
    help: "Evaluate on the test data and save the metrics"
    script:
      - "python -m spacy evaluate ./training/${vars.treebank}/model-best ./corpus/${vars.treebank}/test.spacy --output ./metrics/${vars.treebank}.json --gpu-id ${vars.gpu}"
    deps:
      - "training/${vars.treebank}/model-best"
      - "corpus/${vars.treebank}/test.spacy"
    outputs:
      - "metrics/${vars.treebank}.json"

  - name: package
    help: "Package the trained model so it can be installed"
    script:
      - "python -m spacy package training/${vars.treebank}/model-best packages --name ${vars.package_name} --version ${vars.package_version} --force"
    deps:
      - "training/${vars.treebank}/model-best"
    outputs_no_cache:
      - "packages/${vars.lang}_${vars.package_name}-${vars.package_version}/dist/en_${vars.package_name}-${vars.package_version}.tar.gz"

  - name: clean
    help: "Remove intermediate files"
    script:
      - "rm -rf training/*"
      - "rm -rf metrics/*"
      - "rm -rf corpus/*"

Overwriting tagger_parser_ud/project.yml


In [26]:
!python -m spacy project assets /content/tagger_parser_ud

ℹ Fetching 2 asset(s)
✔ Downloaded asset /content/tagger_parser_ud/assets/vectors.vec.gz
✔ Downloaded asset /content/tagger_parser_ud/assets/UD_Irish-IDT


In [27]:
!python -m spacy project run vectors /content/tagger_parser_ud


================================== vectors ==================================
Running command: /usr/bin/python3 -m spacy init vectors ga assets/vectors.vec.gz corpus/ga_vectors -n ga_fasttext_cc_vectors_md
ℹ Creating blank nlp object for language 'ga'
[2021-11-30 10:39:18,055] [INFO] Reading vectors from assets/vectors.vec.gz
316836it [00:28, 11251.98it/s]
[2021-11-30 10:39:46,415] [INFO] Loaded vectors from assets/vectors.vec.gz
✔ Successfully converted 316836 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/content/tagger_parser_ud/corpus/ga_vectors


In [28]:
%cd /content
!python -m spacy project run preprocess tagger_parser_ud

/content

================================= preprocess =================================
Running command: mkdir -p corpus/UD_Irish-IDT
Running command: /usr/bin/python3 -m spacy convert assets/UD_Irish-IDT/ga_idt-ud-train.conllu corpus/UD_Irish-IDT/ --converter conllu --n-sents 10 --merge-subtokens --morphology
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (401 documents):
corpus/UD_Irish-IDT/ga_idt-ud-train.spacy
Running command: /usr/bin/python3 -m spacy convert assets/UD_Irish-IDT/ga_idt-ud-dev.conllu corpus/UD_Irish-IDT/ --converter conllu --n-sents 10 --merge-subtokens --morphology
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (46 documents):
corpus/UD_Irish-IDT/ga_idt-ud-dev.spacy
Running command: /usr/bin/python3 -m spacy convert assets/UD_Irish-IDT/ga_idt-ud-test.conllu corpus/UD_Irish-IDT/ --converter conllu --n-sents 10 --merge-subtokens --morphology
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (46 do

In [29]:
%%writefile /content/tagger_parser_ud/configs/base_default.cfg
[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "ga"
pipeline = ["tok2vec","tagger","morphologizer","parser"]
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
batch_size = 1000

[components]

[components.morphologizer]
factory = "morphologizer"

[components.morphologizer.model]
@architectures = "spacy.Tagger.v1"
nO = null

[components.morphologizer.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.parser]
factory = "parser"
learn_tokens = false
min_action_freq = 30
moves = null
update_with_oracle_cut_size = 100

[components.parser.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "parser"
extra_state_tokens = false
hidden_width = 128
maxout_pieces = 3
use_upper = true
nO = null

[components.parser.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tagger]
factory = "tagger"

[components.tagger.model]
@architectures = "spacy.Tagger.v1"
nO = null

[components.tagger.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v1"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v1"
width = ${components.tok2vec.model.encode.width}
attrs = ["LOWER","PREFIX","SUFFIX","SHAPE"]
rows = [5000,2500,2500,2500]
include_static_vectors = false

[components.tok2vec.model.encode]
@architectures = "spacy.MaxoutWindowEncoder.v1"
width = 96
depth = 4
window_size = 1
maxout_pieces = 3

[corpora]

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0
gold_preproc = false
limit = 0
augmenter = null

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 2000
gold_preproc = false
limit = 0
augmenter = null

[training]
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"
seed = ${system.seed}
gpu_allocator = ${system.gpu_allocator}
dropout = 0.1
accumulate_gradient = 1
patience = 1600
max_epochs = 0
max_steps = 20000
eval_frequency = 200
frozen_components = []
before_to_disk = null

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2
get_length = null

[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001
t = 0.0

[training.logger]
@loggers = "spacy.ConsoleLogger.v1"
progress_bar = false

[training.optimizer]
@optimizers = "Adam.v1"
beta1 = 0.9
beta2 = 0.999
L2_is_weight_decay = true
L2 = 0.01
grad_clip = 1.0
use_averages = false
eps = 0.00000001
learn_rate = 0.001

[training.score_weights]
morph_per_feat = null
dep_las_per_type = null
sents_p = null
sents_r = null
tag_acc = 0.33
pos_acc = 0.17
morph_acc = 0.17
dep_uas = 0.17
dep_las = 0.17
sents_f = 0.0

[pretraining]

[initialize]
vectors = ${paths.vectors}
init_tok2vec = ${paths.init_tok2vec}
vocab_data = null
lookups = null
before_init = null
after_init = null

[initialize.components]

[initialize.tokenizer]


Writing /content/tagger_parser_ud/configs/base_default.cfg


In [30]:
!rm /content/tagger_parser_ud/configs/default.cfg
!python -m spacy init fill-config /content/tagger_parser_ud/configs/base_default.cfg /content/tagger_parser_ud/configs/default.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/tagger_parser_ud/configs/default.cfg
You can now add your data and train your pipeline:
python -m spacy train default.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [31]:
!python -m spacy project run train tagger_parser_ud


=================================== train ===================================
Running command: /usr/bin/python3 -m spacy train configs/default.cfg --output training/UD_Irish-IDT --gpu-id -1 --paths.train corpus/UD_Irish-IDT/train.spacy --paths.dev corpus/UD_Irish-IDT/dev.spacy --paths.vectors corpus/ga_vectors --nlp.lang=ga
✔ Created output directory: training/UD_Irish-IDT
ℹ Saving to output directory: training/UD_Irish-IDT
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-11-30 10:41:10,449] [INFO] Set up nlp object from config
[2021-11-30 10:41:10,463] [INFO] Pipeline: ['tok2vec', 'tagger', 'morphologizer', 'parser']
[2021-11-30 10:41:10,469] [INFO] Created vocabulary
[2021-11-30 10:41:11,485] [INFO] Added vectors: corpus/ga_vectors
[2021-11-30 10:41:13,027] [INFO] Finished initializing nlp object
[2021-11-30 10:41:25,163] [INFO] Initialized pipeline components: ['tok2vec', 'tagger', 'morphologizer', 'parser']
✔ Initialized pipeline

==

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,931 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155219 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
%%capture
!pip install transformers

In [ ]:
!transformers-cli login
